In [7]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem.AllChem import EmbedMolecule, EmbedMultipleConfs, MMFFOptimizeMoleculeConfs
import pickle
from tqdm import tqdm

In [12]:
def mol_to_morgan(mol, radius=3, n_bits=2048, chiral=False, features=False):
    from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
    return np.frombuffer(
        GetMorganFingerprintAsBitVect(
            mol, radius=radius, nBits=n_bits, useChirality=chiral, useFeatures=features
        ).ToBitString().encode(), "u1") - ord("0")

def mol_to_atoms_coords(
    m, hydrogenate=True, adj_matrix=False, do_morgan=False, optimize=False, numConfs=1, numThreads=1
):
    # Check if m is a string, and if so, convert it to an RDKit molecule object
    if isinstance(m, str):
        m = Chem.MolFromSmiles(m)
        if m is None:
            return None  # Could not parse the SMILES string into a molecule

    m3 = Chem.AddHs(m) if hydrogenate else m
    if optimize and hydrogenate:
        try:
            EmbedMultipleConfs(
                m3, numConfs=numConfs, pruneRmsThresh=0.125, randomSeed=0xF00D, numThreads=numThreads
            )
            opt = MMFFOptimizeMoleculeConfs(m3, mmffVariant="MMFF94s", numThreads=numThreads)
            opt = np.array(opt)
            converged = opt[:, 0] == 0
            if converged.any():
                lowest_eng_conformer = np.argmin(opt[converged][:, 1])
                lowest_energy = opt[converged][lowest_eng_conformer, 1]
                best_conf = np.arange(opt.shape[0])[converged][lowest_eng_conformer]
                c0 = m3.GetConformer(id=int(best_conf))
            else:
                # Fallback in case optimization did not converge
                c0 = m3.GetConformer()
                lowest_energy = None
        except Exception as Ex:
            # Fallback in case of an exception during embedding or optimization
            EmbedMolecule(m3, randomSeed=0xF00D)
            c0 = m3.GetConformers()[-1]
            lowest_energy = None
    else:
        EmbedMolecule(m3, randomSeed=0xF00D)
        c0 = m3.GetConformers()[-1]

    coords = c0.GetPositions()
    atoms = np.array([atom.GetAtomicNum() for atom in m3.GetAtoms()], dtype=np.uint8)

    to_return = [atoms, coords]

    if adj_matrix:
        to_return.append(Chem.GetAdjacencyMatrix(m3))

    if do_morgan:
        to_return.append(mol_to_morgan(m, radius=3, n_bits=2048, chiral=False))

    if optimize:
        to_return.append(lowest_energy)

    return tuple(to_return)

In [9]:
# Load the SMILES data
# Replace 'path_to_your_smiles_data.smiles' with the actual path to your downloaded SMILES file
df_train = pd.read_csv('train_guacamol.smiles', header=None, names=['smiles'])
df_valid = pd.read_csv('valid_guacamol.smiles', header=None, names=['smiles'])
df_test = pd.read_csv('test_guacamol.smiles', header=None, names=['smiles'])

In [14]:
# Preprocess the SMILES data
def preprocess_smiles_data(smiles_data, partition: str):
    preprocessed_data = []
    for smi in tqdm(smiles_data):
        try:
            atoms, coords = mol_to_atoms_coords(smi)
            preprocessed_data.append({
                'smiles': smi,
                'atoms': atoms,
                'coords': coords,
                'partition': partition,
            })
        except Exception as e:
            print(f"Failed to process {smi}: {str(e)}")

    return preprocessed_data

# Save the preprocessed data to a pickle file
with open('train_guacamol.pkl', 'wb') as f:
    pickle.dump(preprocess_smiles_data(df_train['smiles'],'train'), f)

with open('valid_guacamol.pkl', 'wb') as f:
    pickle.dump(preprocess_smiles_data(df_valid['smiles'], 'valid'), f)

with open('test_guacamol.pkl', 'wb') as f:
    pickle.dump(preprocess_smiles_data(df_test['smiles'], 'test'), f)

 62%|██████▏   | 794630/1273104 [18:36:15<32:00:26,  4.15it/s]

Failed to process CCCCCCCCCCCCCC(=O)NC(COC1OC(CO)C(O)C(O)C1O)C(O)CCCCCCCCCCCCC: 'NoneType' object has no attribute 'GetPositions'


 62%|██████▏   | 794830/1273104 [18:36:38<14:50:53,  8.95it/s]

Failed to process CCCCCCCCCCCCCCCC(=O)OCC(COP(=O)(O)OC1C(O)C(O)C(O)C(F)C1O)OC(=O)CCCCCCCCCCCCCCC: 'NoneType' object has no attribute 'GetPositions'


 62%|██████▏   | 795235/1273104 [18:37:19<14:38:57,  9.06it/s]

...

 65%|██████▍   | 822315/1273104 [19:18:01<24:57:39,  5.02it/s]

Failed to process CCCCCCn1ccc(=O)c(O)c1C(=O)NCCN(CCNC(=O)c1c(O)c(=O)ccn1CCCCCC)CCNC(=O)c1c(O)c(=O)ccn1CCCCCC: 'NoneType' object has no attribute 'GetPositions'


 65%|██████▍   | 822598/1273104 [19:18:35<22:37:20,  5.53it/s]

Failed to process CCCCCCCCCCn1c[n+](CCCCCCCCCC)cc1CC(N)C(=O)NC(CCCNC(=N)N)C(N)=O: 'NoneType' object has no attribute 'GetPositions'


 65%|██████▍   | 823575/1273104 [19:20:21<14:39:48,  8.52it/s]

In [ ]:
# Convert the lists of dictionaries to DataFrames and save as CSV files
def save_as_csv(preprocessed_data, filename):
    df = pd.DataFrame(preprocessed_data)
    # If 'coords' and 'atoms' are lists, they will be saved as strings in the CSV. You might need to parse them when reading.
    df.to_csv(filename, index=False)

save_as_csv(preprocessed_train, 'train_guacamol.csv')
save_as_csv(preprocessed_valid, 'valid_guacamol.csv')
save_as_csv(preprocessed_test, 'test_guacamol.csv')